In [16]:
import pandas as pd
import requests
from sklearn.preprocessing import MinMaxScaler
import time
import numpy as np

In [2]:
def to_poi_df(poiType, df):
    result = df[["Name", "Longitude", "Latitude"]]
    result.insert(0, "type", poiType)
    return result
def fetch_google_data(poi):
    fmt = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={name}&inputtype=textquery&locationbias=circle%3A100%40{lat}%2C{lon}&fields=name%2Crating%2Cuser_ratings_total&key={key}"
    r = requests.get(fmt.format(name=poi["Name"], lat=poi["Latitude"], lon=poi["Longitude"], key="MYKEY"))
    json = r.json()
    return json["candidates"][0]["user_ratings_total"] if "candidates" in json and len(json["candidates"])>0 and "user_ratings_total" in json["candidates"][0] else 0

In [3]:
poi_data = to_poi_df("activity", pd.read_csv("Activities.csv"))
poi_data = poi_data.append(to_poi_df("accommodation", pd.read_csv("Accommodation.csv")))
poi_data = poi_data.append(to_poi_df("attraction", pd.read_csv("Attractions.csv")))
significance = []
for i in range(poi_data.shape[0]):
    if i%100==0:
        print("Updating {}".format(i))
    repeat = True
    while repeat:
        try:
            significance.append(fetch_google_data(poi_data.iloc[i]))
            repeat = False
        except:
            print("Re-trying {}".format(i))
            time.sleep(5)
            

Updating 0
Re-trying 27
Updating 100
Re-trying 113
Re-trying 190
Updating 200
Re-trying 267
Updating 300
Re-trying 344
Updating 400
Re-trying 421
Re-trying 498
Updating 500
Re-trying 575
Updating 600
Re-trying 652
Updating 700
Re-trying 729
Updating 800
Re-trying 806
Re-trying 883
Updating 900
Re-trying 960
Updating 1000
Re-trying 1037
Updating 1100
Re-trying 1116
Re-trying 1193
Updating 1200
Updating 1300
Updating 1400
Updating 1500
Re-trying 1560
Updating 1600
Re-trying 1637
Updating 1700
Re-trying 1714
Re-trying 1791
Updating 1800
Re-trying 1868
Updating 1900
Re-trying 1945
Updating 2000
Re-trying 2022
Re-trying 2099
Updating 2100
Re-trying 2176
Updating 2200
Re-trying 2253
Updating 2300
Re-trying 2330
Updating 2400
Re-trying 2407
Updating 2500
Updating 2600
Updating 2700
Updating 2800
Re-trying 2833
Updating 2900
Re-trying 2910
Updating 3000
Updating 3100
Updating 3200
Updating 3300
Updating 3400
Updating 3500
Updating 3600
Updating 3700
Updating 3800
Updating 3900
Re-trying 3972
U

In [4]:
poi_data["significance"] = significance
poi_data.to_csv("poi.tsv", sep="\t")

In [21]:
poi_data["scaled_sig"] = poi_data["significance"].clip(0, 2000)
poi_data["scaled_sig"] = MinMaxScaler(feature_range=(0,100))\
    .fit_transform(poi_data["scaled_sig"].values.T.reshape(-1,1))\
    .astype(int)
poi_data.to_csv("poi_scaledsig.tsv", sep="\t")